# Species Prior
PaCMAP repo: https://github.com/YingfanWang/PaCMAP

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from plantclef.spark import get_spark

spark = get_spark()
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/14 01:17:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/14 01:17:22 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
25/04/14 01:17:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### embeddings

In [3]:
import os
from pathlib import Path

# Get list of stored filed in cloud bucket
root = Path(os.path.expanduser("~"))
! date

Mon Apr 14 01:17:24 AM EDT 2025


### test embeddings

In [4]:
# Path and dataset names
data_path = f"{root}/p-dsgt_clef2025-0/shared/plantclef/data/embeddings"

# Define the path to the train and test parquet files
test_path = f"{data_path}/test_2025/test_2025_embed_logits"

# Read the parquet files into a spark DataFrame
test_df = spark.read.parquet(test_path)

# Show the data
test_df.printSchema()
test_df.show(n=5)

root
 |-- image_name: string (nullable = true)
 |-- output: struct (nullable = true)
 |    |-- cls_token: array (nullable = true)
 |    |    |-- element: float (containsNull = true)
 |    |-- logits: array (nullable = true)
 |    |    |-- element: float (containsNull = true)
 |-- sample_id: integer (nullable = true)



+--------------------+--------------------+---------+
|          image_name|              output|sample_id|
+--------------------+--------------------+---------+
|CBN-Pla-A1-201908...|{[0.47354543, 1.5...|        0|
|CBN-Pla-D6-201908...|{[-0.39621377, 1....|        0|
|CBN-PdlC-C5-20140...|{[-0.5331654, 0.2...|        0|
|LISAH-BOU-0-37-20...|{[1.2480925, 0.47...|        0|
|CBN-Pla-E4-201308...|{[0.7065191, 1.70...|        0|
+--------------------+--------------------+---------+
only showing top 5 rows



In [5]:
# count number of rows: should be 2105 for grid=1x1
print(f"Number of rows: {test_df.count()}")

Number of rows: 2105


In [6]:
regions = [
    "2024-CEV3",
    "CBN-can",
    "CBN-PdlC",
    "CBN-Pla",
    "CBN-Pyr",
    "GUARDEN-AMB",
    "GUARDEN-CBNMed",
    "LISAH-BOU",
    "LISAH-BVD",
    "LISAH-JAS",
    "LISAH-PEC",
    "OPTMix",
    "RNNB",
]

len(regions)

13

In [7]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, regexp_extract


def prepare_emb_df(df: DataFrame, embed_col: str = "output.cls_token") -> DataFrame:
    """
    Prepare the DataFrame by renaming the embedding column and selecting relevant columns.
    """
    regex_pattern = "|".join([f"^{region}" for region in regions])
    # add the grouped_regions column using regexp_extract
    test_df_with_regions = df.withColumn(
        "grouped_regions", regexp_extract(col("image_name"), f"({regex_pattern})", 1)
    )

    # check if there are any unmatched regions (empty strings)
    unmatched_count = test_df_with_regions.filter(col("grouped_regions") == "").count()
    print(f"Number of rows with unmatched regions: {unmatched_count}")

    # show the result with the new column
    test_df_with_regions.select("image_name", "grouped_regions", embed_col).show(
        10, truncate=True
    )

    # Count occurrences of each region
    region_counts = (
        test_df_with_regions.groupBy("grouped_regions")
        .count()
        .orderBy(col("count").desc())
    )
    region_counts.show(20)
    return test_df_with_regions


test_df_with_regions = prepare_emb_df(test_df, "output.cls_token")

Number of rows with unmatched regions: 0
+--------------------+---------------+--------------------+
|          image_name|grouped_regions|           cls_token|
+--------------------+---------------+--------------------+
|CBN-Pla-A1-201908...|        CBN-Pla|[0.47354543, 1.55...|
|CBN-Pla-D6-201908...|        CBN-Pla|[-0.39621377, 1.2...|
|CBN-PdlC-C5-20140...|       CBN-PdlC|[-0.5331654, 0.21...|
|LISAH-BOU-0-37-20...|      LISAH-BOU|[1.2480925, 0.478...|
|CBN-Pla-E4-201308...|        CBN-Pla|[0.7065191, 1.709...|
|CBN-PdlC-D6-20150...|       CBN-PdlC|[-0.32394692, 0.4...|
|CBN-PdlC-F2-20170...|       CBN-PdlC|[1.4019761, 1.783...|
|CBN-PdlC-A6-20180...|       CBN-PdlC|[-0.49399343, 1.1...|
|RNNB-3-12-2023051...|           RNNB|[-0.37940657, 0.1...|
|CBN-PdlC-F4-20150...|       CBN-PdlC|[-0.26687536, 1.2...|
+--------------------+---------------+--------------------+
only showing top 10 rows

+---------------+-----+
|grouped_regions|count|
+---------------+-----+
|       CBN-PdlC|  81

In [8]:
test_df_with_regions.count()

2105

In [9]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
# import pacmap

# # Create a clustering pipeline
# clustering_pipeline = Pipeline(
#     [
#         ("scaler", StandardScaler()),
#         ("pacmap", pacmap.PaCMAP(n_components=2, random_state=42)),
#         (
#             "cluster",
#             KMeans(n_clusters=5, random_state=42),
#         ),
#     ]
# )


In [10]:
import pickle

# model directory
model_dir = (
    Path(os.path.expanduser("~")) / "p-dsgt_clef2025-0/shared/plantclef/models/pacmap"
)
pipeline_filename = model_dir / "plant_clustering_pipeline.pkl"


# load the clustering model
with open(pipeline_filename, "rb") as file:
    clustering_pipeline = pickle.load(file)

AttributeError: Can't get attribute 'PaCMAPTransformer' on <module '__main__'>

In [ ]:
import numpy as np

# Convert to Pandas DF
col_name = "output.cls_token"
df = test_df_with_regions.select([col_name, "grouped_regions"])
pandas_df = df.select(["cls_token", "grouped_regions"]).toPandas()

# Fit and predict clusters
embeddings = np.stack(pandas_df["cls_token"].values)
labels = clustering_pipeline.fit_predict(embeddings)

# Attach results to the DataFrame
pandas_df["cluster"] = labels
pandas_df["pacmap_1"] = clustering_pipeline.named_steps["pacmap"].embedding_[:, 0]
pandas_df["pacmap_2"] = clustering_pipeline.named_steps["pacmap"].embedding_[:, 1]